In [1]:

import numpy as np
import pandas as pd
import tqdm
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


import spacy
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from spacy.lang.fr.examples import sentences 
import re 


import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

from gensim.models.wrappers import LdaMallet

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from gensim.models import TfidfModel

import matplotlib.pyplot as plt

C:\Users\Admin\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by

## Preparing the Data

In [3]:
df = pd.read_excel("shortEditionPosts.xlsx")
df

,title,genres,post,date,time,reads,likes,compet
0,Petits monstres,['Horreur'],Les poissons avaient cessé de mordre d'un seul...,2021-12-31 09:30:26,2,277\n,16,En compétition
1,L'Atelier de Maman,"['Instant de vie', 'Arts']","Quand on y entre, on aurait presque envie, d'a...",2021-12-27 09:30:05,2,532\n,22,En compétition
2,Une nouvelle chaussée,"['Instant de vie', 'Drame']","Du plat de la main, Achille caressa les cambru...",2021-12-26 09:30:04,3,283\n,26,En compétition
3,Ma vie où l'on voit la mer,"['Instant de vie', 'Société']",La plupart des gens ont une vie où l'on ne voi...,2021-12-24 09:30:25,3,667\n,57,En compétition
4,Juste un moment chez Sonia,"['Instant de vie', 'Humour']",Sonia adore ces soirées où elle invite des gen...,2021-12-22 09:30:05,3,1 027\n,57,En compétition
...,...,...,...,...,...,...,...,...
5765,Trois ballons,"['Nature', 'Suspense']","Trois ballons. Un rouge, un vert, un orange.\n...",2011-03-23 15:47:14,1,780\n,9,Qualifié
5766,La gardienne,"['Nature', 'Merveilleux']",Regardez un immense champ vert... Un champ ple...,2011-03-23 15:46:33,1,1 598\n,7,Qualifié
5767,Départ pour un calvaire,"['Famille', 'Instant de vie']","Ce matin, je me lève avec un poids insoutenabl...",2011-03-23 15:43:10,1,992\n,23,Qualifié
5768,Morceau classique,"['Instant de vie', 'Arts']",Madame se mit à jouer du piano. Ses doigts cou...,2011-03-23 10:00:00,1,4 141\n,123,Lauréat


In [4]:
textes = df.drop(columns=['title', 'date', 'time','reads','likes','compet','genres'], axis=1)
textes

,post
0,Les poissons avaient cessé de mordre d'un seul...
1,"Quand on y entre, on aurait presque envie, d'a..."
2,"Du plat de la main, Achille caressa les cambru..."
3,La plupart des gens ont une vie où l'on ne voi...
4,Sonia adore ces soirées où elle invite des gen...
...,...
5765,"Trois ballons. Un rouge, un vert, un orange.\n..."
5766,Regardez un immense champ vert... Un champ ple...
5767,"Ce matin, je me lève avec un poids insoutenabl..."
5768,Madame se mit à jouer du piano. Ses doigts cou...


In [5]:
data = textes['post']

print(data[0]) 

Les poissons avaient cessé de mordre d'un seul coup... À la surface de l'étang, il régnait un calme anormal. Aucune vaguelette, aucune bulle venue du fond, comme si la vie avait cessé brutalement là-dessous... Tom et son frère Jules savaient très bien ce que cela signifiait...  — Passe-moi la boite, s'il te plaît ! dit Tom. — Tiens ! — Non pas celle-ci, donne-moi les petits rouges. Ils sont plus nerveux. Tom ouvrit le couvercle de la petite boite et découvrit avec satisfaction un amas de vers grouillants qui s'agitaient dans tous les sens. — Regarde comme ils sont beaux. Avec ça, je vais faire un malheur, annonça-t-il avec certitude à son frère. Il fouilla avec ses doigts la sciure, il sentit alors les petites bêtes lui chatouiller le creux de sa main. Il aimait vraiment cette sensation. Il prit tout son temps pour choisir l'asticot idéal. Celui auquel aucun poisson ne résisterait. Tom et Jules s'étaient spécialisés dans l'élevage des asticots. Les petits, les gros, les joufflus, les g

In [6]:
def clean(data):
    liste = []
    for text in data:
        test = text.lower()
        test = re.sub(r"[0-9]+", "", test)
        REMOVE_PUNCT = re.compile("[—.;:!\?,\"()\[\]]")
        test = REMOVE_PUNCT.sub("", test)
        test = re.sub(r"\b\w{1,2}\b", "", test)
        REMOVE_PUNCT_ = re.compile("[']")
        test = REMOVE_PUNCT_.sub("", test)
        REMOVE_PUNCT_1 = re.compile("[-]")
        test = REMOVE_PUNCT_1.sub(" ", test)
        REMOVE_PUNCT_1 = re.compile("[«]")
        test = REMOVE_PUNCT_1.sub("", test)
        REMOVE_PUNCT_1 = re.compile("[»]")
        test = REMOVE_PUNCT_1.sub("", test)
        liste.append(test)
    data = pd.Series(liste)
    return data
data = clean(data)
print(data[0])

les poissons avaient cessé  mordre  seul coup   surface  étang  régnait  calme anormal aucune vaguelette aucune bulle venue  fond comme   vie avait cessé brutalement  dessous tom  son frère jules savaient très bien  que cela signifiait   passe moi  boite   plaît  dit tom  tiens   non pas celle  donne moi les petits rouges ils sont plus nerveux tom ouvrit  couvercle   petite boite  découvrit avec satisfaction  amas  vers grouillants qui agitaient dans tous les sens  regarde comme ils sont beaux avec   vais faire  malheur annonça   avec certitude  son frère  fouilla avec ses doigts  sciure  sentit alors les petites bêtes lui chatouiller  creux   main  aimait vraiment cette sensation  prit tout son temps pour choisir asticot idéal celui auquel aucun poisson  résisterait tom  jules étaient spécialisés dans élevage des asticots les petits les gros les joufflus les gras les nerveux les colorés rien  manquait  leur panoplie ils avaient jusqu  jour tout étudié tout essayé enfin presque ils ava

In [7]:
stopwords_ = stopwords.words("French")

In [8]:
print (stopwords_)
print(len(stopwords_))

['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 'avez', 'ont', 'aurai', 'auras', 'aura', 'aurons', 'aur

In [9]:
def delete_stopwords(data):
    text_out = []
    for texte_ in data:
        texte = ''.join(str(e) for e in texte_)
        mots = texte.split()
        final_mots = []
        for mot in mots:
            if mot not in stopwords_:
                final_mots.append(mot)
            
        text_out.append(' '.join(str(mot) for mot in final_mots))
    data = pd.Series(text_out)
    return(data)
data = delete_stopwords(data)

In [10]:
print(data[0])

poissons cessé mordre seul coup surface étang régnait calme anormal aucune vaguelette aucune bulle venue fond comme vie cessé brutalement dessous tom frère jules savaient très bien cela signifiait passe boite plaît dit tom tiens non celle donne petits rouges plus nerveux tom ouvrit couvercle petite boite découvrit satisfaction amas vers grouillants agitaient tous sens regarde comme beaux vais faire malheur annonça certitude frère fouilla doigts sciure sentit alors petites bêtes chatouiller creux main aimait vraiment cette sensation prit tout temps choisir asticot idéal celui auquel aucun poisson résisterait tom jules spécialisés élevage asticots petits gros joufflus gras nerveux colorés rien manquait panoplie jusqu jour tout étudié tout essayé enfin presque commencé faire pourrir poisson celui pêché étang ensuite mis idée récupérer tout trouvaient restes alimentaires oiseaux morts couleuvres rats chiens écrasés rien perdait tout transformait recyclage devenu affaire technique simple di

In [92]:
!pip install spacy
!python -m spacy download fr_core_news_sm


C:\Users\Hp\OneDrive\Bureau\MES_PROJETS\TopicModelling>doskey make=mingw32-make.exe 

C:\Users\Hp\OneDrive\Bureau\MES_PROJETS\TopicModelling>doskey make=mingw32-make.exe 
[+] Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [11]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("fr_core_news_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)
lemmatized_texts = lemmatization(data)

In [12]:
lemmatized_texts[0]

'poisson cesser mordr seul coup étang régner calme anormal vaguelette bulle venir fondre vie cesser brutalement dessou tom frère savoir très bien signifier passer boite plaît dire tom tiens non donne petit rouge plus nerveux tom ouvrir couvercle petit boite découvrir satisfaction amas grouillant agiter tout sens regarde beal aller faire malheur frère sciure sentit alors petit bête chatouiller creux main aimer vraiment sensation prendre temps choisir asticot idéal celui auquel poisson résister tom jule spécialisé élevage asticot petit gros joufflus gras nerveux colorer manquer panoplie jusqu jour étudié tout essayer enfin presque commencer pourrir poisson celui pêcher étang ensuite mettre ider récupérer trouvaient reste alimentaire oiseal mort couleuvre rat chien écraser perdre tout transformer recyclage devenir affaire technique simple disposer soin cadavre grille branche poirier ensuite attendaient putréfaction travail récupérer plaçaient dessou tamis grand bassine effet jolie bestiol

In [13]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['poisson', 'cesser', 'mordr', 'seul', 'coup', 'etang', 'regner', 'calme', 'anormal', 'vaguelette', 'bulle', 'venir', 'fondre', 'vie', 'cesser', 'brutalement', 'dessou', 'tom', 'frere', 'savoir']


In [14]:
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)


C:\Users\Admin\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [15]:
#examples de bigrams et de trigrams
liste = []
for texte in data_bigrams_trigrams:
    for mot in texte:
        if "_" in mot and mot not in liste:
            liste.append(mot)
print(liste)

['blouse_blanc', 'asseoir_tailleur', 'quelque_part', 'rue_pietonne', 'marteau_piqueur', 'galerie_marchand', 'porte_vitrer', 'racle_gorge', 'valoir_mieux', 'tant_pis', 'eclater_rir', 'poser_question', 'developpement_personnel', 'week_end', 'nul_part', 'feu_artific', 'rythme_effrener', 'pension_alimentaire', 'per_noel', 'peu_import', 'ang_gardien', 'pere_noel', 'battement_cœur', 'prince_charmer', 'laurier_rose', 'appel_secour', 'bonjour_monsieur', 'commun_mortel', 'systeme_solaire', 'voie_lacter', 'etoile_filante', 'garder_precieusement', 'froid_mordre', 'herbe_hauter', 'goutte_goutte', 'vieil_homme', 'adresser_parol', 'feux_artifice', 'douce_melodi', 'rendre_compte', 'goutte_plui', 'rechauffement_climatique', 'un_autre', 'jusqu_epuisemer', 'frisson_parcourut', 'nuit_etoiler', 'surgo_pedago', 'distributeur_automatique', 'casser_croute', 'goutte_sueur', 'allumer_cigarette', 'pol_emploi', 'tant_ginette', 'crise_cardiaque', 'boite_conserve', 'battre_chamade', 'courant_air', 'nid_douillet', 

In [16]:
id2word = corpora.Dictionary(data_bigrams_trigrams)
def TF_IDF(data):
    texts = data

    corpus = [id2word.doc2bow(text) for text in texts]

    tfidf = TfidfModel(corpus, id2word=id2word)


    words  = []
    words_missing_in_tfidf = []
    for i in range(len (corpus)):
        bow = corpus[i]
        low_value_words = [] 
        tfidf_ids = [id for id, value in tfidf[bow]]
        bow_ids = [id for id, value in bow]
        values =[value for id, value in tfidf[bow]]
        low_value = np.median(values)
        low_value_words = [id for id, value in tfidf[bow] if value < low_value]
        drops = low_value_words+words_missing_in_tfidf
        for item in drops:
            words.append(id2word[item])
        words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] 

        new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
        corpus[i] = new_bow
    return(corpus)
corpus = TF_IDF(data_bigrams_trigrams)

In [17]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1],
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.6)


## Vizualizing the Data

In [18]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

C:\Users\Admin\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.181618  0.351866       1        1  15.635833
3      0.134888 -0.377437       2        1  13.396540
0      0.147982  0.391403       3        1  11.792737
9     -0.222123 -0.329744       4        1  11.666120
2     -0.316672  0.144638       5        1  10.352698
6      0.141851  0.193772       6        1   9.174365
7      0.400560 -0.126978       7        1   8.675306
8     -0.343419 -0.087002       8        1   7.799075
1      0.263037 -0.000013       9        1   5.861829
4     -0.024485 -0.160504      10        1   5.645497, topic_info=        Term         Freq        Total Category  logprob  loglift
923       an  1282.000000  1282.000000  Default  30.0000  30.0000
1244   fille  1245.000000  1245.000000  Default  29.0000  29.0000
139     mere  1188.000000  1188.000000  Default  28.0000  28.0000
152      non   950.000000   950.000000  Default  27.0000  27.0000
112    homme  1144.000000  1144.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
76      donc   221.643959   535.223971  Topic10  -5.3580   1.9927
790      fai   188.417334   427.861307  Topic10  -5.5204   2.0542
3168    truc   121.293926   194.418706  Topic10  -5.9609   2.4025
1735  laisse   144.315103   357.723888  Topic10  -5.7871   1.9665
4331   bouge   113.207013   176.017715  Topic10  -6.0299   2.4329

[489 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
1816       4  0.996312   acheter
14411      2  0.993893    acteur
1          9  0.996489   affaire
1817       2  0.037134       age
1817       6  0.961774       age
...      ...       ...       ...
243        3  0.110950   vouloir
243        4  0.865001   vouloir
243        6  0.022601   vouloir
245        4  0.997578  vraiment
5566       9  0.989587     œuvre

[651 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 1, 10, 3, 7, 8, 9, 2, 5])

In [19]:
# afficher les mots clés pour les 10 thèmes
print(lda_model.print_topics())

[(0, '0.011*"porte" + 0.007*"chambre" + 0.007*"main" + 0.006*"lit" + 0.006*"noir" + 0.005*"levre" + 0.005*"rouge" + 0.005*"blanc" + 0.005*"corps" + 0.005*"penser"'), (1, '0.017*"grand" + 0.016*"heure" + 0.004*"gauche" + 0.004*"affaire" + 0.004*"maitre" + 0.004*"objet" + 0.003*"manqu" + 0.003*"toil" + 0.003*"princesse" + 0.003*"scene"'), (2, '0.013*"homme" + 0.012*"vie" + 0.011*"amour" + 0.009*"peur" + 0.007*"maison" + 0.007*"aimer" + 0.007*"frere" + 0.006*"souvenir" + 0.006*"rose" + 0.005*"fleur"'), (3, '0.007*"train" + 0.005*"monsieur" + 0.003*"travail" + 0.003*"premier" + 0.003*"ordinaire" + 0.002*"place" + 0.002*"semaine" + 0.002*"donc" + 0.002*"jeune" + 0.002*"femme"'), (4, '0.019*"non" + 0.015*"oui" + 0.012*"mort" + 0.010*"bon" + 0.010*"con" + 0.006*"devoir" + 0.005*"donc" + 0.005*"soldat" + 0.005*"guerre" + 0.004*"fai"'), (5, '0.006*"eau" + 0.004*"terre" + 0.003*"sable" + 0.003*"route" + 0.003*"soleil" + 0.003*"sol" + 0.003*"marche" + 0.003*"odeur" + 0.002*"voiture" + 0.002*"sens

In [20]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

coherence_model_lda = CoherenceModel(model=lda_model, texts=data_bigrams_trigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -10.138657604057437

Coherence Score:  0.3158041588529496


In [21]:
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_bigrams_trigrams, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [38]:
grid = {}
grid['Validation_Set'] = {}
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
beta = list(np.arange(0.01, 1, 0.3))
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    for i in range(len(corpus_sets)):    
        for k in topics_range:   
            for a in alpha:
                for b in beta:
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()



  0%|                                                                                          | 0/540 [01:15<?, ?it/s]


  0%|▏                                                                               | 1/540 [00:54<8:06:51, 54.20s/it]

  0%|▎                                                                               | 2/540 [01:59<9:04:06, 60.68s/it]

  1%|▍                                                                               | 3/540 [02:49<8:20:04, 55.87s/it]

  1%|▌                                                                               | 4/540 [03:35<7:44:30, 52.00s/it]

  1%|▋                                                                               | 5/540 [04:24<7:33:10, 50.82s/it]

  1%|▉                                                                               | 6/540 [05:19<7:44:51, 52.23s/it]

  1%|█                                                                               | 7/540 [06:13<7:48:14, 52.71s/it]

  1%|█▏                      

 24%|█████████████████▊                                                         | 128/540 [10:40:12<8:14:39, 72.04s/it]

 24%|█████████████████▉                                                         | 129/540 [10:41:15<7:54:03, 69.21s/it]

 24%|██████████████████                                                         | 130/540 [10:42:28<8:00:33, 70.33s/it]

 24%|██████████████████▏                                                        | 131/540 [10:43:43<8:09:59, 71.88s/it]

 24%|██████████████████▎                                                        | 132/540 [10:44:45<7:48:58, 68.97s/it]

 25%|██████████████████▍                                                        | 133/540 [10:45:50<7:38:21, 67.57s/it]

 25%|██████████████████▌                                                        | 134/540 [10:46:59<7:40:38, 68.08s/it]

 25%|██████████████████▊                                                        | 135/540 [10:48:14<7:53:34, 70.16s/it]

 25%|██████████████████▉        

 47%|███████████████████████████████████▌                                       | 256/540 [13:22:14<6:19:59, 80.28s/it]

 48%|███████████████████████████████████▋                                       | 257/540 [13:23:28<6:10:06, 78.47s/it]

 48%|███████████████████████████████████▊                                       | 258/540 [13:24:38<5:57:14, 76.01s/it]

 48%|███████████████████████████████████▉                                       | 259/540 [13:25:42<5:39:30, 72.49s/it]

 48%|████████████████████████████████████                                       | 260/540 [13:26:58<5:43:05, 73.52s/it]

 48%|████████████████████████████████████▎                                      | 261/540 [13:28:38<6:18:31, 81.40s/it]

 49%|████████████████████████████████████▍                                      | 262/540 [13:29:47<6:00:12, 77.74s/it]

 49%|████████████████████████████████████▌                                      | 263/540 [13:30:51<5:39:33, 73.55s/it]

 49%|███████████████████████████

 71%|█████████████████████████████████████████████████████▎                     | 384/540 [16:25:10<4:11:58, 96.92s/it]

 71%|█████████████████████████████████████████████████████▍                     | 385/540 [16:26:49<4:11:49, 97.48s/it]

 71%|████████████████████████████████████████████████████▉                     | 386/540 [16:28:44<4:24:22, 103.00s/it]

 72%|█████████████████████████████████████████████████████▊                     | 387/540 [16:30:01<4:02:06, 94.94s/it]

 72%|█████████████████████████████████████████████████████▉                     | 388/540 [16:31:21<3:49:28, 90.58s/it]

 72%|██████████████████████████████████████████████████████                     | 389/540 [16:32:38<3:37:53, 86.58s/it]

 72%|██████████████████████████████████████████████████████▏                    | 390/540 [16:34:01<3:33:49, 85.53s/it]

 72%|█████████████████████████████████████████████████████▌                    | 391/540 [16:36:32<4:20:38, 104.96s/it]

 73%|███████████████████████████

 95%|████████████████████████████████████████████████████████████████████████    | 512/540 [20:06:06<52:58, 113.50s/it]

 95%|████████████████████████████████████████████████████████████████████████▏   | 513/540 [20:07:35<47:44, 106.10s/it]

 95%|█████████████████████████████████████████████████████████████████████████▎   | 514/540 [20:08:59<43:06, 99.48s/it]

 95%|█████████████████████████████████████████████████████████████████████████▍   | 515/540 [20:10:38<41:23, 99.33s/it]

 96%|████████████████████████████████████████████████████████████████████████▌   | 516/540 [20:12:37<42:06, 105.27s/it]

 96%|█████████████████████████████████████████████████████████████████████████▋   | 517/540 [20:14:04<38:13, 99.73s/it]

 96%|█████████████████████████████████████████████████████████████████████████▊   | 518/540 [20:15:32<35:18, 96.30s/it]

 96%|██████████████████████████████████████████████████████████████████████████   | 519/540 [20:16:55<32:13, 92.08s/it]

 96%|███████████████████████████

In [40]:
hyperparams = pd.read_csv("lda_tuning_results.csv")
hyperparams

,Validation_Set,Topics,Alpha,Beta,Coherence
0,75% Corpus,2,0.01,0.01,0.282723
1,75% Corpus,2,0.01,0.31,0.303086
2,75% Corpus,2,0.01,0.61,0.310328
3,75% Corpus,2,0.01,0.9099999999999999,0.291748
4,75% Corpus,2,0.01,symmetric,0.323348
...,...,...,...,...,...
535,100% Corpus,10,asymmetric,0.01,0.328123
536,100% Corpus,10,asymmetric,0.31,0.512130
537,100% Corpus,10,asymmetric,0.61,0.469631
538,100% Corpus,10,asymmetric,0.9099999999999999,0.384623


In [53]:
sorted_coherence = np.sort(hyperparams["Coherence"])

In [42]:
hyperparams[hyperparams["Coherence"]==np.max(hyperparams["Coherence"])]

,Validation_Set,Topics,Alpha,Beta,Coherence
143,75% Corpus,6,symmetric,0.9099999999999999,0.551874


In [23]:
best_num_topic = 6
best_alpha = "symmetric"
best_beta = 0.9099999999999999
compute_coherence_values(corpus=corpus, dictionary=id2word, 
                                                  k=best_num_topic, a=best_alpha, b=best_beta)

0.510702558244516

In [24]:
def LDA_modele(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    return(lda_model)

In [25]:
lda_model_final = LDA_modele (corpus=corpus, dictionary=id2word, 
                                                  k=best_num_topic, a=best_alpha, b=best_beta)

In [26]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model_final, corpus, id2word, mds="mmds", R=30)
vis

C:\Users\Admin\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.030134  0.166039       1        1  50.766732
3      0.156356  0.080078       2        1  47.046191
1     -0.056664 -0.057264       3        1   0.667993
0     -0.039210 -0.066635       4        1   0.630673
5     -0.045047 -0.058710       5        1   0.492128
4     -0.045569 -0.063508       6        1   0.396284, topic_info=            Term         Freq        Total Category  logprob  loglift
144     monsieur  1185.000000  1185.000000  Default  30.0000  30.0000
71          dire   987.000000   987.000000  Default  29.0000  29.0000
923           an  1137.000000  1137.000000  Default  28.0000  28.0000
152          non   881.000000   881.000000  Default  27.0000  27.0000
104        grand   114.000000   114.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
21192  aubergine     0.335572    10.259451   Topic6  -9.1945   2.1107
8652   expansion     0.344179    19.647696   Topic6  -9.1692   1.4862
5184       bulbe     0.304011    12.598542   Topic6  -9.2933   1.8065
36022     spleen     0.277401    12.520124   Topic6  -9.3849   1.7212
28103       soul     0.248246     9.940986   Topic6  -9.4960   1.8408

[341 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
1492       1  0.010718   accord
1492       2  0.986043   accord
1816       1  0.004678  acheter
1816       2  0.991708  acheter
1817       1  0.059177      age
...      ...       ...      ...
56419      1  0.606956  zygomat
56419      2  0.202319  zygomat
56419      5  0.202319  zygomat
50134      1  0.382460      đao
50134      2  0.382460      đao

[701 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 2, 1, 6, 5])

In [27]:
print('\nPerplexity: ', lda_model_final.log_perplexity(corpus))


Perplexity:  -9.78413203972083


### Recommandation

In [28]:
new_text=pd.read_excel("new_text.xlsx")
new_text=clean(new_text['text'])
new_text=delete_stopwords(new_text)
new_text=lemmatization(new_text)
new_text=gen_words(new_text)
text_liked=new_text[2]
pred=lda_model_final[id2word.doc2bow(text_liked)]
m=0
c=-1
for i in pred:
    if i[1]>m:
        c=i[0]
        m=i[1]
        
print("le texte apparient au cluster {} avec la probabilité {}".format(c,m))    
pred

le texte apparient au cluster 3 avec la probabilité 0.7363701462745667


[(1, 0.04911127), (2, 0.21110573), (3, 0.73637015)]

In [29]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [30]:
df_topic_sents_keywords = format_topics_sentences(lda_model_final, corpus, textes['post'])

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,2.0,0.7482,"petit, homme, eau, main, noir, mer, terre, bla...",Les poissons avaient cessé de mordre d'un seul...
1,1,2.0,0.8179,"petit, homme, eau, main, noir, mer, terre, bla...","Quand on y entre, on aurait presque envie, d'a..."
2,2,2.0,0.6820,"petit, homme, eau, main, noir, mer, terre, bla...","Du plat de la main, Achille caressa les cambru..."
3,3,3.0,0.8548,"monsieur, avai, an, dire, non, oui, heure, fem...",La plupart des gens ont une vie où l'on ne voi...
4,4,3.0,0.9048,"monsieur, avai, an, dire, non, oui, heure, fem...",Sonia adore ces soirées où elle invite des gen...
5,5,3.0,0.7090,"monsieur, avai, an, dire, non, oui, heure, fem...","Ce n'est pas que Germaine était jolie, mais il..."
6,6,2.0,0.5672,"petit, homme, eau, main, noir, mer, terre, bla...",Si la grande course hippique annuelle sur mètr...
7,7,3.0,0.7829,"monsieur, avai, an, dire, non, oui, heure, fem...","Pourquoi tu ris ? — Pour rien. — Oh, c'est déj..."
8,8,3.0,0.6047,"monsieur, avai, an, dire, non, oui, heure, fem...",Pressée. Comme toujours. Le dernier client vie...
9,9,3.0,0.9963,"monsieur, avai, an, dire, non, oui, heure, fem...","Face au pharaonique rayon bio du magasin, je s..."


In [31]:
same_topic=df_dominant_topic[df_dominant_topic['Dominant_Topic']==c]

In [32]:
def documentSimilarity(filename_1, filename_2): 
    sorted_word_list_1 = word_frequencies_for_file(filename_1) 
    sorted_word_list_2 = word_frequencies_for_file(filename_2) 
    distance = vector_angle(sorted_word_list_1, sorted_word_list_2) 
    return(distance)
      

In [33]:
def word_frequencies_for_file(word_list):     
    freq_mapping = count_frequency(word_list)   
    return freq_mapping

In [34]:
def count_frequency(word_list):     
    D = {}       
    for new_word in word_list:         
        if new_word in D: 
            D[new_word] = D[new_word] + 1            
        else: 
            D[new_word] = 1              
    return D 

In [35]:
import math
def vector_angle(D1, D2):  
    numerator = dotProduct(D1, D2) 
    denominator = math.sqrt(dotProduct(D1, D1)*dotProduct(D2, D2))       
    return math.acos(numerator / denominator) 

def dotProduct(D1, D2):  
    Sum = 0.0     
    for key in D1:           
        if key in D2: 
            Sum += (D1[key] * D2[key])               
    return Sum

In [36]:
distances=[]
for i in range(len(same_topic)):
    distances.append(documentSimilarity(text_liked, data_bigrams_trigrams[i]))
min_dist=min(distances)
sugg=textes['post'][distances.index(min_dist)]
print('le texte suggéré est:\n ', sugg)
                   

le texte suggéré est:
  Je ne suis plus mucophage depuis l'âge de 12 ans environ, je pense à ça car Grand-père est actuellement en train de fouiller dans son nez avec son petit doigt, les yeux dans le vague. C'était les obsèques de Grand-mère ce matin, alors Grand-père est complètement perdu. Il ne sait pas quoi faire avec ses mains, il n'y a pas si longtemps, il s'en servait encore pour frapper Grand-mère, on dirait que ça lui manque déjà. L'année dernière, à 83 ans, il lui avait encore cassé un doigt.Grand-mère a été incinérée, sa dernière volonté était que Grand-père, quand son tour viendra, ne puisse pas être enterré à côté d'elle.  Pour se changer les idées, Grand-père veut prendre la voiture et aller se boire une bière chez son copain de toujours Lucien. Je lui rappelle qu'il est atteint de narcolepsie, qu'il s'endort au volant, qu'il n'a plus d'assurance car il a causé trois accidents ces six derniers mois et que Lucien est mort l'année dernière. Sur ce, il me répond :— Ha ouais

### Kmeans

In [37]:
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
fr=list(fr_stop)
vectorizer = TfidfVectorizer(stop_words=fr)
X = vectorizer.fit_transform(list(data))
X.shape

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['qu', 'quelqu'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(5770, 86626)

In [38]:
true_k = 6
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X[:5769,:])

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=6, n_init=1, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [39]:
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

In [40]:
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print('%s'% terms[ind])

Cluster 0:
maman
papa
petit
petite
père
aime
ans
faire
fille
mère
Cluster 1:
terre
nuit
vent
ciel
mer
vie
hommes
temps
eau
jour
Cluster 2:
mère
père
vie
ans
jamais
grand
jour
amour
temps
fois
Cluster 3:
yeux
porte
temps
main
faire
sais
regarde
tête
fois
homme
Cluster 4:
soleil
yeux
eau
ciel
homme
temps
main
corps
petit
regard
Cluster 5:
faire
temps
oui
vie
fois
jamais
bon
ans
petit
jour


In [41]:
predicted_cluster=model.predict(X[5769,:])
print("the predicted cluster is",predicted_cluster[0])


the predicted cluster is 5


In [42]:
l=list(model.labels_)
l.append(predicted_cluster[0])
df['kmean cluster']=l
df

,title,genres,post,date,time,reads,likes,compet,kmean cluster
0,Petits monstres,['Horreur'],Les poissons avaient cessé de mordre d'un seul...,2021-12-31 09:30:26,2,277\n,16,En compétition,5
1,L'Atelier de Maman,"['Instant de vie', 'Arts']","Quand on y entre, on aurait presque envie, d'a...",2021-12-27 09:30:05,2,532\n,22,En compétition,4
2,Une nouvelle chaussée,"['Instant de vie', 'Drame']","Du plat de la main, Achille caressa les cambru...",2021-12-26 09:30:04,3,283\n,26,En compétition,5
3,Ma vie où l'on voit la mer,"['Instant de vie', 'Société']",La plupart des gens ont une vie où l'on ne voi...,2021-12-24 09:30:25,3,667\n,57,En compétition,5
4,Juste un moment chez Sonia,"['Instant de vie', 'Humour']",Sonia adore ces soirées où elle invite des gen...,2021-12-22 09:30:05,3,1 027\n,57,En compétition,5
...,...,...,...,...,...,...,...,...,...
5765,Trois ballons,"['Nature', 'Suspense']","Trois ballons. Un rouge, un vert, un orange.\n...",2011-03-23 15:47:14,1,780\n,9,Qualifié,1
5766,La gardienne,"['Nature', 'Merveilleux']",Regardez un immense champ vert... Un champ ple...,2011-03-23 15:46:33,1,1 598\n,7,Qualifié,4
5767,Départ pour un calvaire,"['Famille', 'Instant de vie']","Ce matin, je me lève avec un poids insoutenabl...",2011-03-23 15:43:10,1,992\n,23,Qualifié,2
5768,Morceau classique,"['Instant de vie', 'Arts']",Madame se mit à jouer du piano. Ses doigts cou...,2011-03-23 10:00:00,1,4 141\n,123,Lauréat,1


In [43]:
same_cluster_kmean=df[df['kmean cluster']==predicted_cluster[0]]


In [44]:
distances_kmean=[]
for i in range(len(same_cluster_kmean)-1):
    distances_kmean.append(documentSimilarity(data_bigrams_trigrams[5769], data_bigrams_trigrams[i]))
min_dist_kmean=min(distances_kmean)
sugg_kmean=textes['post'][distances_kmean.index(min_dist_kmean)]
print('le texte suggéré est:\n ', sugg_kmean)

le texte suggéré est:
  Un matin, il était là. Énorme, noir des pattes jusqu’au bec, le corps luisant et les plumes lissées… Il se tenait sur la plus haute branche de l’arbre, juste en face de ma chambre située au premier étage. Je ne le remarquai d’abord pas vraiment, rien de plus qu’une ombre fugace sur le bord extérieur de ma vision. Et puis les choses changèrent, petit à petit. Chaque jour en me réveillant, je le découvrais sur son perchoir, serein, attendant je ne sais quoi. Il m’observait fixement sans montrer une once de lassitude et dès que nos regards se croisaient, il me saluait d’un vif croassement ou d’un hochement de bec.  Le temps passant, il se rapprochait de plus en plus de la vitre et je m’inquiétais moins de sa sombre et mystérieuse présence. De fait, avant que je n’en prenne conscience, nous étions entrés tous les deux dans une certaine routine. Dès qu’il m’apercevait, il s’annonçait d’un cri bref et je lui répondais poliment d’un timide « bonjour » ou d’un geste de 

In [45]:
df['genres'][distances_kmean.index(min_dist_kmean)]

"['Instant de vie', 'Suspense', 'Nature']"

In [46]:
df['genres'][5769]

"['Drame', 'Amitié', 'Instant de vie']"

Tous les deux partagent le genre Instant de vie.